In [59]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

dev = qml.device("default.qubit", wires=["player1", "player2", "goalie"])


# Put any extra functions you want here

def calculate_rotation_angle_gamma(gamma, c):
    alpha = np.arccos(gamma)
    p = gamma ** 2
    new_p = c * p
    new_alpha = np.arccos(np.sqrt(new_p))
    return new_alpha - alpha

def calculate_rotation_angle_delta(delta, c):
    alpha = np.arcsin(delta)
    p = delta ** 2
    new_p = c * p
    new_alpha = np.arcsin(np.sqrt(new_p))
    return alpha - new_alpha

def both_bits_the_same(gamma, delta, x, y, z, wires):
    rot = calculate_rotation_angle_delta(delta, z)
    qml.RY(-2 * rot, wires=wires)

def one_zero(gamma, delta, x, y, z, wires):
    rot = calculate_rotation_angle_gamma(gamma, x)
    qml.RY(2 * rot, wires=wires)

def zero_one(gamma, delta, x, y, z, wires):
    rot = calculate_rotation_angle_gamma(gamma, y)
    qml.RY(2 * rot, wires=wires)

def state_prep(player_coeffs, goalie_coeffs):
    """
    Contains quantum operations that prepare |psi> and |phi>. We recommend using
    qml.StatePrep!

    Args:
        - player_coeffs (list(float)): 
            The coefficients, alpha, beta, and kappa (in that order) that describe 
            the quantum state of players 1 and 2:

            |psi> = alpha|01> + beta|10> + kappa(|00> + |11>)

        - goalie_coeffs (list(float)):
            The coefficients, gamma and delta (in that order) that describe 
            the quantum state of the goalie:

            |phi> = gamma|0> + delta|1>
    """


    alpha, beta, kappa = player_coeffs
    gamma, delta = goalie_coeffs

    # Put your code here #
    qml.StatePrep([kappa, alpha, beta, kappa], wires=["player1", "player2"])
    qml.StatePrep([gamma, delta], wires="goalie")

@qml.qnode(dev)
def save_percentage(player_coeffs, goalie_coeffs, x, y, z):
    """
    Calculates the save percentage of the goalie.

    NOTE: This QNode may only contain 7 operations or less (counting any 
    operations used in the state_prep function) and must use three conditional
    measurements (i.e., 3 instances of qml.cond).

    Args:
        - player_coeffs (list(float)): 
            The coefficients, alpha, beta, and kappa (in that order) that describe 
            the quantum state of players 1 and 2:

            |psi> = alpha|01> + beta|10> + kappa(|00> + |11>)

        - goalie_coeffs (list(float)):
            The coefficients, gamma and delta (in that order) that describe 
            the quantum state of the goalie:

            |phi> = gamma|0> + delta|1>
        
        - x, y, z (float): 
            The amounts that affect the goalie's save percentage based on 
            measuring the players.

    Returns:
        - (numpy.tensor): The save percentage of the goalie.
    """
    state_prep(player_coeffs, goalie_coeffs)
    gamma, delta = goalie_coeffs

    # Put your code here #
    m1 = qml.measure(wires="player1")
    m2 = qml.measure(wires="player2")
    qml.cond(m1 & ~m2, one_zero)(gamma, delta, x, y, z, wires="goalie")
    qml.cond(~m1 & m2, zero_one)(gamma, delta, x, y, z, wires="goalie")
    qml.cond((m1 & m2) | (~m1 & ~m2), both_bits_the_same)(gamma, delta, x, y, z, wires="goalie")

    return qml.probs(wires="goalie")



In [60]:
save_percentage([0.74199663, 0.17932039, 0.45677413], [0.28034464, 0.95989941], 0.999, 0.99, 0.98)

In one_zero
alpha: 1.2866431987817415 p: 0.0785931171767296 new_p: 0.07851452405955286 new_alpha: 1.2867892603261062 rot: 0.0001460615443646951
In zero one
alpha: 1.2866431987817415 p: 0.0785931171767296 new_p: 0.0778071860049623 new_alpha: 1.2881068368884034 rot: 0.0014636381066619464
In both_bits_the_same
0.9214068773183481 0.9029787397719812 1.254043727714753


tensor([0.08584768, 0.91415233], requires_grad=True)

In [51]:
(0.28034464 ** 2) * 0.9

0.07073380545905664

In [61]:

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    player_coeffs, goalie_coeffs, x, y, z = ins
    output = save_percentage(player_coeffs, goalie_coeffs, x, y, z).tolist()
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    sp = solution_output
    _sp = json.loads(expected_output)
    print(f"Solution: {sp}, Expected: {_sp}")
    ops = save_percentage.tape._ops
    num_ops = len(ops)
    num_cond = [op.name for op in ops].count('Conditional')
    names = [op.name for op in ops]
    state_prep_check = ('StatePrep' or 'MottonenStatePreparation' or 'AmplitudeEmbedding') in names

    assert np.allclose(sp, _sp, rtol=1e-4), "Your calculated save percentage is incorrect."
    assert num_ops < 8, "You used more than 7 operations in your save_percentage function."
    assert num_ops > 2, "You definitely need more than 2 operations..."
    assert state_prep_check, "You can use StatePrep, MottonenStatePreparation, or AmplitudeEmbedding to prepare states."
    assert num_cond == 3, "You haven't used exactly 3 qml.cond operators."


# These are the public test cases
test_cases = [
    ('[[0.74199663, 0.17932039, 0.45677413], [0.28034464, 0.95989941], 0.999, 0.99, 0.98]', '[0.08584767923415959, 0.9141523336414634]'),
    ('[[0.09737041, 0.40230525, 0.64368839], [0.00111111, 0.99999938], 0.9, 0.95, 0.92]', '[0.06629469110239884, 0.9337053066603161]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.74199663, 0.17932039, 0.45677413], [0.28034464, 0.95989941], 0.999, 0.99, 0.98]'...
In one_zero
alpha: 1.2866431987817415 p: 0.0785931171767296 new_p: 0.07851452405955286 new_alpha: 1.2867892603261062 rot: 0.0001460615443646951
In zero one
alpha: 1.2866431987817415 p: 0.0785931171767296 new_p: 0.0778071860049623 new_alpha: 1.2881068368884034 rot: 0.0014636381066619464
In both_bits_the_same
0.9214068773183481 0.9029787397719812 1.254043727714753
Solution: [0.08584767898167227, 0.9141523338939507], Expected: [0.08584767923415959, 0.9141523336414634]
Correct!
Running test case 1 with input '[[0.09737041, 0.40230525, 0.64368839], [0.00111111, 0.99999938], 0.9, 0.95, 0.92]'...
In one_zero
alpha: 1.5696852165662736 p: 1.2345654321e-06 new_p: 1.11110888889e-06 new_alpha: 1.5697422351003978 rot: 5.70185341242091e-05
In zero one
alpha: 1.5696852165662736 p: 1.2345654321e-06 new_p: 1.172837160495e-06 new_alpha: 1.569713350516758 rot: 2.813395048439915e-05
In b